In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder

from sklearn.preprocessing import MinMaxScaler

%config IPCompleter.greedy = True

In [88]:
ruta_proyecto = r'C:\Users\manug\Desktop\Data Science\DataSets\Wall-Mart\wallmart\\'

In [89]:
nombre_cat = 'cat_resultado_eda.pickle'
nombre_num = 'num_resultado_eda.pickle'

In [90]:
cat = pd.read_pickle(ruta_proyecto + '02_Datos/03_Trabajo/' + nombre_cat)
num = pd.read_pickle(ruta_proyecto + '02_Datos/03_Trabajo/' + nombre_num)

In [91]:
df = pd.concat([cat,num], axis = 1)
df

,isholiday,type,store,temperature,fuel_price,markdown1,markdown2,markdown3,markdown4,markdown5,cpi,unemployment,size,dept,weekly_sales
date,,,,,,,,,,,,,,,
2010-02-05,False,A,1,42.31,2.572,0.00,0.00,0.00,0.00,0.0,211.096358,8.106,151315,1,24924.50
2010-02-05,False,A,1,42.31,2.572,0.00,0.00,0.00,0.00,0.0,211.096358,8.106,151315,2,50605.27
2010-02-05,False,A,1,42.31,2.572,0.00,0.00,0.00,0.00,0.0,211.096358,8.106,151315,3,13740.12
2010-02-05,False,A,1,42.31,2.572,0.00,0.00,0.00,0.00,0.0,211.096358,8.106,151315,4,39954.04
2010-02-05,False,A,1,42.31,2.572,0.00,0.00,0.00,0.00,0.0,211.096358,8.106,151315,5,32229.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-07-20,False,B,45,78.89,3.567,6682.75,24.24,12.83,2249.85,2288.3,191.167043,8.684,118221,93,3568.64
2012-07-20,False,B,45,78.89,3.567,6682.75,24.24,12.83,2249.85,2288.3,191.167043,8.684,118221,94,3618.52
2012-07-20,False,B,45,78.89,3.567,6682.75,24.24,12.83,2249.85,2288.3,191.167043,8.684,118221,95,53136.34


## Creación de nuevas variables
* Lags y ventanas m

#### Creación lags

In [92]:
lags = [1, 2, 3, 4]
for lag in lags:
    df[f'weekly_sales_lag_{lag}'] = df.groupby(['store', 'dept'])['weekly_sales'].shift(lag)
    df[f'temperature_lag_{lag}'] = df.groupby(['store'])['temperature'].shift(lag)
    df[f'fuel_price_lag_{lag}'] = df.groupby(['store'])['fuel_price'].shift(lag)
    df[f'cpi_lag_{lag}'] = df.groupby(['store'])['cpi'].shift(lag)
    df[f'unemployment_lag_{lag}'] = df.groupby(['store'])['unemployment'].shift(lag)
    df[f'isholiday_lag_{lag}'] = df.groupby(['store'])['isholiday'].shift(lag)

#### Creación medias movil

In [93]:
windows = [2, 4, 8] 

for window in windows:
    df[f'weekly_sales_mm_{window}'] = df.groupby(['store', 'dept'])['weekly_sales'].transform(lambda x: x.rolling(window).mean())
    df[f'temperature_mm_{window}'] = df.groupby(['store'])['temperature'].transform(lambda x: x.rolling(window).mean())
    df[f'fuel_price_mm_{window}'] = df.groupby(['store'])['fuel_price'].transform(lambda x: x.rolling(window).mean())
    df[f'cpi_mm_{window}'] = df.groupby(['store'])['cpi'].transform(lambda x: x.rolling(window).mean())
    df[f'unemployment_mm_{window}'] = df.groupby(['store'])['unemployment'].transform(lambda x: x.rolling(window).mean())

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 380107 entries, 2010-02-05 to 2012-07-20
Data columns (total 54 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   isholiday           380107 non-null  object 
 1   type                380107 non-null  object 
 2   store               380107 non-null  int64  
 3   temperature         380107 non-null  float64
 4   fuel_price          380107 non-null  float64
 5   markdown1           380107 non-null  float64
 6   markdown2           380107 non-null  float64
 7   markdown3           380107 non-null  float64
 8   markdown4           380107 non-null  float64
 9   markdown5           380107 non-null  float64
 10  cpi                 380107 non-null  float64
 11  unemployment        380107 non-null  float64
 12  size                380107 non-null  int64  
 13  dept                380107 non-null  int64  
 14  weekly_sales        380107 non-null  float64
 15  weekly_sales_lag_1

#### Eliminación de nuevos nulos generados

In [95]:
df.dropna(inplace=True)

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 357384 entries, 2010-03-26 to 2012-07-20
Data columns (total 54 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   isholiday           357384 non-null  object 
 1   type                357384 non-null  object 
 2   store               357384 non-null  int64  
 3   temperature         357384 non-null  float64
 4   fuel_price          357384 non-null  float64
 5   markdown1           357384 non-null  float64
 6   markdown2           357384 non-null  float64
 7   markdown3           357384 non-null  float64
 8   markdown4           357384 non-null  float64
 9   markdown5           357384 non-null  float64
 10  cpi                 357384 non-null  float64
 11  unemployment        357384 non-null  float64
 12  size                357384 non-null  int64  
 13  dept                357384 non-null  int64  
 14  weekly_sales        357384 non-null  float64
 15  weekly_sales_lag_1

#### Identicicación de la target

In [97]:
target = df.weekly_sales

#### Divición cat y num

In [98]:
cat = df.select_dtypes(include = 'O')
num = df.select_dtypes(exclude = 'O')

## Transformación de categóricas

# One Hot Encoding

In [99]:
cat

,isholiday,type,isholiday_lag_1,isholiday_lag_2,isholiday_lag_3,isholiday_lag_4
date,,,,,,
2010-03-26,False,A,False,False,False,False
2010-03-26,False,A,False,False,False,False
2010-03-26,False,A,False,False,False,False
2010-03-26,False,A,False,False,False,False
2010-03-26,False,A,False,False,False,False
...,...,...,...,...,...,...
2012-07-20,False,B,False,False,False,False
2012-07-20,False,B,False,False,False,False
2012-07-20,False,B,False,False,False,False


In [103]:
var_ohe = ['isholiday', 'type', 'isholiday_lag_1', 'isholiday_lag_2',
       'isholiday_lag_3', 'isholiday_lag_4']

In [104]:
ohe = OneHotEncoder(sparse_output=False,handle_unknown='ignore')

In [105]:
cat_ohe = ohe.fit_transform(cat[var_ohe])

In [106]:
cat_ohe

array([[1., 0., 1., ..., 0., 1., 0.],
       [1., 0., 1., ..., 0., 1., 0.],
       [1., 0., 1., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.]])

In [107]:
cat_ohe = pd.DataFrame(cat_ohe, columns=ohe.get_feature_names_out())

In [108]:
cat_ohe

,isholiday_False,isholiday_True,type_A,type_B,type_C,isholiday_lag_1_False,isholiday_lag_1_True,isholiday_lag_2_False,isholiday_lag_2_True,isholiday_lag_3_False,isholiday_lag_3_True,isholiday_lag_4_False,isholiday_lag_4_True
0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
357379,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
357380,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
357381,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
357382,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


### Target Encoding

In [109]:
var_te = ['isholiday', 'type', 'isholiday_lag_1', 'isholiday_lag_2',
       'isholiday_lag_3', 'isholiday_lag_4']

In [110]:
te = TargetEncoder(min_samples_leaf=100, return_df=False)

In [111]:
pd.set_option('future.no_silent_downcasting', True)
cat_te = te.fit_transform(cat[var_te], y=target)

In [112]:
nombre_te = [variable + '_te' for variable in var_te]

In [113]:
cat_te = pd.DataFrame(cat_te, columns=nombre_te)

In [114]:
cat_te

,isholiday_te,type_te,isholiday_lag_1_te,isholiday_lag_2_te,isholiday_lag_3_te,isholiday_lag_4_te
0,16041.276077,20259.231187,16038.423529,16037.772302,16038.506551,16039.291241
1,16041.276077,20259.231187,16038.423529,16037.772302,16038.506551,16039.291241
2,16041.276077,20259.231187,16038.423529,16037.772302,16038.506551,16039.291241
3,16041.276077,20259.231187,16038.423529,16037.772302,16038.506551,16039.291241
4,16041.276077,20259.231187,16038.423529,16037.772302,16038.506551,16039.291241
...,...,...,...,...,...,...
357379,16041.276077,12344.416155,16038.423529,16037.772302,16038.506551,16039.291241
357380,16041.276077,12344.416155,16038.423529,16037.772302,16038.506551,16039.291241
357381,16041.276077,12344.416155,16038.423529,16037.772302,16038.506551,16039.291241
357382,16041.276077,12344.416155,16038.423529,16037.772302,16038.506551,16039.291241


### Reescalar Variables

#### Min-Max Scaler

#### Variables a reescalar

In [115]:
num.columns

Index(['store', 'temperature', 'fuel_price', 'markdown1', 'markdown2',
       'markdown3', 'markdown4', 'markdown5', 'cpi', 'unemployment', 'size',
       'dept', 'weekly_sales', 'weekly_sales_lag_1', 'temperature_lag_1',
       'fuel_price_lag_1', 'cpi_lag_1', 'unemployment_lag_1',
       'weekly_sales_lag_2', 'temperature_lag_2', 'fuel_price_lag_2',
       'cpi_lag_2', 'unemployment_lag_2', 'weekly_sales_lag_3',
       'temperature_lag_3', 'fuel_price_lag_3', 'cpi_lag_3',
       'unemployment_lag_3', 'weekly_sales_lag_4', 'temperature_lag_4',
       'fuel_price_lag_4', 'cpi_lag_4', 'unemployment_lag_4',
       'weekly_sales_mm_2', 'temperature_mm_2', 'fuel_price_mm_2', 'cpi_mm_2',
       'unemployment_mm_2', 'weekly_sales_mm_4', 'temperature_mm_4',
       'fuel_price_mm_4', 'cpi_mm_4', 'unemployment_mm_4', 'weekly_sales_mm_8',
       'temperature_mm_8', 'fuel_price_mm_8', 'cpi_mm_8', 'unemployment_mm_8'],
      dtype='object')

In [116]:
target = df['weekly_sales']

In [124]:
store = df['store']

In [117]:
variables_incluir_mms = ['temperature', 'fuel_price', 'markdown1', 'markdown2', 'markdown3',
       'markdown4', 'markdown5', 'cpi', 'unemployment', 'size', 'dept',
       'weekly_sales_lag_1', 'temperature_lag_1',
       'fuel_price_lag_1', 'cpi_lag_1', 'unemployment_lag_1',
       'weekly_sales_lag_2', 'temperature_lag_2', 'fuel_price_lag_2',
       'cpi_lag_2', 'unemployment_lag_2', 'weekly_sales_lag_3',
       'temperature_lag_3', 'fuel_price_lag_3', 'cpi_lag_3',
       'unemployment_lag_3', 'weekly_sales_lag_4', 'temperature_lag_4',
       'fuel_price_lag_4', 'cpi_lag_4', 'unemployment_lag_4',
       'weekly_sales_mm_2', 'temperature_mm_2', 'fuel_price_mm_2', 'cpi_mm_2',
       'unemployment_mm_2', 'weekly_sales_mm_4', 'temperature_mm_4',
       'fuel_price_mm_4', 'cpi_mm_4', 'unemployment_mm_4', 'weekly_sales_mm_8',
       'temperature_mm_8', 'fuel_price_mm_8', 'cpi_mm_8', 'unemployment_mm_8']

In [118]:
mms = MinMaxScaler()

df_mms = mms.fit_transform(df[variables_incluir_mms])

In [119]:
nombres_mms = [variable + '_mms' for variable in variables_incluir_mms]
df_mms = pd.DataFrame(df_mms, columns=nombres_mms)
df_mms

,temperature_mms,fuel_price_mms,markdown1_mms,markdown2_mms,markdown3_mms,markdown4_mms,markdown5_mms,cpi_mms,unemployment_mms,size_mms,...,weekly_sales_mm_4_mms,temperature_mm_4_mms,fuel_price_mm_4_mms,cpi_mm_4_mms,unemployment_mm_4_mms,weekly_sales_mm_8_mms,temperature_mm_8_mms,fuel_price_mm_8_mms,cpi_mm_8_mms,unemployment_mm_8_mms
0,0.523581,0.122006,0.000000,0.002536,0.000205,0.000000,0.000000,0.852754,0.393611,0.630267,...,0.069901,0.546551,0.116992,0.854242,0.393611,0.102919,0.550379,0.116156,0.854490,0.393611
1,0.523581,0.122006,0.000000,0.002536,0.000205,0.000000,0.000000,0.852754,0.393611,0.630267,...,0.133360,0.538894,0.118663,0.853746,0.393611,0.167232,0.546551,0.116992,0.854242,0.393611
2,0.523581,0.122006,0.000000,0.002536,0.000205,0.000000,0.000000,0.852754,0.393611,0.630267,...,0.033576,0.531238,0.120334,0.853250,0.393611,0.042149,0.542723,0.117827,0.853994,0.393611
3,0.523581,0.122006,0.000000,0.002536,0.000205,0.000000,0.000000,0.852754,0.393611,0.630267,...,0.104786,0.523581,0.122006,0.852754,0.393611,0.131159,0.538894,0.118663,0.853746,0.393611
4,0.523581,0.122006,0.000000,0.002536,0.000205,0.000000,0.000000,0.852754,0.393611,0.630267,...,0.080878,0.523581,0.122006,0.852754,0.393611,0.100838,0.535066,0.119499,0.853498,0.393611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357379,0.792074,0.587187,0.075386,0.002768,0.000296,0.033344,0.021087,0.653493,0.450078,0.451136,...,0.016792,0.792074,0.587187,0.653493,0.450078,0.018471,0.792074,0.587187,0.653493,0.450078
357380,0.792074,0.587187,0.075386,0.002768,0.000296,0.033344,0.021087,0.653493,0.450078,0.451136,...,0.012421,0.792074,0.587187,0.653493,0.450078,0.013434,0.792074,0.587187,0.653493,0.450078
357381,0.792074,0.587187,0.075386,0.002768,0.000296,0.033344,0.021087,0.653493,0.450078,0.451136,...,0.171109,0.792074,0.587187,0.653493,0.450078,0.209947,0.792074,0.587187,0.653493,0.450078
357382,0.792074,0.587187,0.075386,0.002768,0.000296,0.033344,0.021087,0.653493,0.450078,0.451136,...,0.023781,0.792074,0.587187,0.653493,0.450078,0.027779,0.792074,0.587187,0.653493,0.450078


## Se unifican los datasets transformados

In [125]:
df_tablon = pd.concat([cat_ohe, cat_te, df_mms, target.reset_index(), store.reset_index()], axis=1)

In [126]:
df_tablon

,isholiday_False,isholiday_True,type_A,type_B,type_C,isholiday_lag_1_False,isholiday_lag_1_True,isholiday_lag_2_False,isholiday_lag_2_True,isholiday_lag_3_False,...,unemployment_mm_4_mms,weekly_sales_mm_8_mms,temperature_mm_8_mms,fuel_price_mm_8_mms,cpi_mm_8_mms,unemployment_mm_8_mms,date,weekly_sales,date,store
0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.393611,0.102919,0.550379,0.116156,0.854490,0.393611,2010-03-26,26229.21,2010-03-26,1
1,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.393611,0.167232,0.546551,0.116992,0.854242,0.393611,2010-03-26,41892.55,2010-03-26,1
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.393611,0.042149,0.542723,0.117827,0.853994,0.393611,2010-03-26,10366.85,2010-03-26,1
3,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.393611,0.131159,0.538894,0.118663,0.853746,0.393611,2010-03-26,33872.07,2010-03-26,1
4,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.393611,0.100838,0.535066,0.119499,0.853498,0.393611,2010-03-26,38971.89,2010-03-26,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357379,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.450078,0.018471,0.792074,0.587187,0.653493,0.450078,2012-07-20,3568.64,2012-07-20,45
357380,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.450078,0.013434,0.792074,0.587187,0.653493,0.450078,2012-07-20,3618.52,2012-07-20,45
357381,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.450078,0.209947,0.792074,0.587187,0.653493,0.450078,2012-07-20,53136.34,2012-07-20,45
357382,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.450078,0.027779,0.792074,0.587187,0.653493,0.450078,2012-07-20,6902.68,2012-07-20,45


## Guardar df_tablon

In [127]:
ruta_df_tablon = ruta_proyecto + '02_Datos/03_Trabajo/' + 'df_tablon.pickle'

In [128]:
df_tablon.to_pickle(ruta_df_tablon)

In [129]:
df_tablon.store

0          1
1          1
2          1
3          1
4          1
          ..
357379    45
357380    45
357381    45
357382    45
357383    45
Name: store, Length: 357384, dtype: int64